In [1]:
import argparse
import os
import time
import shutil

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
     

import torchvision
import torchvision.transforms as transforms
import sys

sys.path.insert(1, '../../software')
from models import *


In [2]:
#send an input and grap the value by using prehook like HW3


In [3]:
weight_int2 = torch.randint(low=-8, high=8, size=torch.Size([16,16,3,3])).float() # w_int can be calculated by weight_q and w_delta
#weight_int2 = (-8 * torch.ones(torch.Size([16,16,3,3]))).float() # w_int can be calculated by weight_q and w_delta
#print(weight_int) # you should see clean integer numbers

In [4]:
x_int2 = torch.randint(low=0,high=4, size=torch.Size([128,16,4,4])).float()
#x_int2 = (3 * torch.ones(torch.Size([128,16,4,4]))).float()
#print(x_int) # you should see clean integer numbers 

In [5]:
conv_int2 = torch.nn.Conv2d(in_channels = 8, out_channels=8, kernel_size = 3, padding=1, bias = False)
conv_int2.weight = torch.nn.parameter.Parameter(weight_int2)

output_int2 =  conv_int2.forward(x_int2)    # output_int can be calculated with conv_int and x_int
relu = torch.nn.ReLU()
relu_output_int2 = relu(output_int2)
#print(output_recovered)

In [6]:


# act_int.size = torch.Size([128, 64, 32, 32])  <- batch_size, input_ch, ni, nj
a_int = x_int2[0,:,:,:]  # pick only one input out of batch
# a_int.size() = [64, 32, 32]

# conv_int.weight.size() = torch.Size([64, 64, 3, 3])  <- output_ch, input_ch, ki, kj
w_int = torch.reshape(weight_int2, (weight_int2.size(0), weight_int2.size(1), -1))  # merge ki, kj index to kij
# w_int.weight.size() = torch.Size([64, 64, 9])
                      
padding = 1
stride = 1
row_num = 16 # row and column number
col_num = 8

nig = range(a_int.size(1))  ## ni group
njg = range(a_int.size(2))  ## nj group

icg = range(int(w_int.size(1)))  ## input channel 
ocg = range(int(w_int.size(0)))  ## output channel

ic_tileg = range(int(len(icg)/row_num))
oc_tileg = range(int(len(ocg)/col_num))

kijg = range(w_int.size(2))
ki_dim = int(math.sqrt(w_int.size(2)))  ## Kernel's 1 dim size

######## Padding before Convolution #######
a_pad = torch.zeros(len(icg), len(nig)+padding*2, len(nig)+padding*2).cuda()
# a_pad.size() = [64, 32+2pad, 32+2pad]
a_pad[ :, padding:padding+len(nig), padding:padding+len(njg)] = a_int.cuda()
a_pad = torch.reshape(a_pad, (a_pad.size(0), -1))
# a_pad.size() = [64, (32+2pad)*(32+2pad)]


a_tile = torch.zeros(len(ic_tileg), row_num,    a_pad.size(1)).cuda() 
w_tile = torch.zeros(len(oc_tileg)*len(ic_tileg), col_num, row_num, len(kijg)).cuda() 

for ic_tile in ic_tileg:
    a_tile[ic_tile,:,:] = a_pad[ic_tile*row_num:(ic_tile+1)*row_num,:]

for ic_tile in ic_tileg:
    for oc_tile in oc_tileg:
        w_tile[ic_tile*len(oc_tileg) + oc_tile,:,:,:] = w_int[oc_tile*col_num:(oc_tile+1)*col_num, ic_tile*row_num:(ic_tile+1)*row_num, :]



###########################################

p_nijg = range(a_pad.size(1)) ## psum nij group

psum = torch.zeros(len(ic_tileg), len(oc_tileg), col_num, len(p_nijg), len(kijg)).cuda() 

for kij in kijg:
    for ic_tile in ic_tileg:       # Tiling into array_sizeXarray_size array
        for oc_tile in oc_tileg:   # Tiling into array_sizeXarray_size array        
            for nij in p_nijg:       # time domain, sequentially given input
                    m = nn.Linear(col_num, row_num, bias=False)
                    #m.weight = torch.nn.Parameter(w_int[oc_tile*array_size:(oc_tile+1)*array_size, ic_tile*array_size:(ic_tile+1)*array_size, kij])
                    m.weight = torch.nn.Parameter(w_tile[len(oc_tileg)*ic_tile+oc_tile,:,:,kij])
                    psum[ic_tile, oc_tile, :, nij, kij] = m(a_tile[ic_tile,:,nij]).cuda()
 

In [7]:
import math

a_pad_ni_dim = int(math.sqrt(a_pad.size(1))) # 32

o_ni_dim = int((a_pad_ni_dim - (ki_dim- 1) - 1)/stride + 1)
o_nijg = range(o_ni_dim**2)    

print(len(o_nijg))

out = torch.zeros(len(ocg), len(o_nijg)).cuda()
  
   
### SFP accumulation ###
for o_nij in o_nijg: 
    for kij in kijg:
        for ic_tile in ic_tileg:    
            for oc_tile in oc_tileg:   
                out[oc_tile*col_num:(oc_tile+1)*col_num, o_nij] = out[oc_tile*col_num:(oc_tile+1)*col_num, o_nij] + \
                psum[ic_tile, oc_tile, :, int(o_nij/o_ni_dim)*a_pad_ni_dim + o_nij%o_ni_dim + int(kij/ki_dim)*a_pad_ni_dim + kij%ki_dim, kij]
                ## 4th index = (int(o_nij/30)*32 + o_nij%30) + (int(kij/3)*32 + kij%3)

16


In [8]:
### show this cell partially. The following cells should be printed by students ###
tile_id = 0 
nij = 200 # just a random number
X = a_tile[tile_id,:,:]  # [tile_num, array row num, time_steps]

bit_precision = 2
file = open('P2_Files/activation.txt', 'w') #write to file
file.write('#time0row7[msb-lsb],time0row6[msb-lst],....,time0row0[msb-lst]#\n')
file.write('#time1row7[msb-lsb],time1row6[msb-lst],....,time1row0[msb-lst]#\n')
file.write('#................#\n')

for i in range(X.size(1)):  # time step
    for j in range(X.size(0)): # row #
        X_bin = '{0:02b}'.format(round(X[X.size(0)-1-j,i].item()))
        for k in range(bit_precision):
            file.write(X_bin[k])        
        #file.write(' ')  # for visibility with blank between words, you can use
    file.write('\n')
file.close() #close file    


In [9]:
### Complete this cell ###
kij = 0
array_row_num = 8

bit_precision = 4
for tile_id in range(2):
    for kij in range(9):
        W = w_tile[tile_id,:,:,kij]  # w_tile[tile_num, array col num, array row num, kij]
        file = open('P2_Files/Tile' + str(tile_id) + '/weight_' + str(kij) + '.txt', 'w') #write to file
        file.write('#col0row14[msb-lsb],col0row12[msb-lst],....,col0row0[msb-lst]#\n')
        file.write('#col0row15[msb-lsb],col0row13[msb-lst],....,col0row1[msb-lst]#\n')
#        file.write('#col1row7[msb-lsb],col1row6[msb-lst],....,col1row0[msb-lst]#\n')
        file.write('#................#\n')
        
        for i in range(W.size(0)):  # col
            for m in range(2):  
                for j in range(array_row_num): # row #
                    
                #print(W.size(1)-2*(j+1)+m)
                    W_bin = '{0:04b}'.format(round(W[i,W.size(1)-2*(j+1)+m].item()) + (16 if (round(W[i,W.size(1)-2*(j+1)+m].item()) < 0) else 0))
                    for k in range(bit_precision):
                        file.write(W_bin[k])        
                    #file.write(' ')  # for visibility with blank between words, you can use
                file.write('\n')
        file.close() #close file   
print(W.size())

torch.Size([8, 16])


In [10]:
### Complete this cell ###
ic_tile_id = 0

kij = 0
nij = 200
# psum[len(ic_tileg), len(oc_tileg), array_size, len(p_nijg), len(kijg)]


bit_precision = 16

for oc_tile_id in range(2):
    for kij in range(9):
        psum_tile = psum[ic_tile_id,oc_tile_id,:,:,kij]
        file = open('P2_Files/Tile' + str(oc_tile_id) + '/psum_' + str(kij) + '.txt', 'w') #write to file
        file.write('#time0col7[msb-lsb],time0col6[msb-lst],....,time0col0[msb-lst]#\n')
        file.write('#time1col7[msb-lsb],time1col6[msb-lst],....,time1col0[msb-lst]#\n')
        file.write('#................#\n')
    
        for i in range(psum_tile.size(1)):  # time step
            for j in range(psum_tile.size(0)): # col #
                #psum_bin = '{0:016b}'.format(round(psum_tile[psum_tile.size(0)-1-j,i].item()) + (2**bit_precision if (round(psum_tile[psum_tile.size(0)-1-j,i].item()) < 0) else 0))
                curr_psum = round(psum_tile[psum_tile.size(0)-1-j,i].item())
                #if (i == 7 and kij == 0):
                #    print(curr_psum)
                if (curr_psum < 0):
                    #if (i == 7 and kij == 0):
                    #    print('{0:016b}'.format(curr_psum))
                    curr_psum = curr_psum + 2**bit_precision
                    #if (i == 7 and kij == 0):
                    #    print('{0:016b}'.format(curr_psum))
                psum_bin = '{0:016b}'.format(curr_psum)
                
                for k in range(bit_precision):
                    file.write(psum_bin[k])        
                #file.write(' ')  # for visibility with blank between words, you can use
            file.write('\n')
        file.close() #close file 

In [11]:
### Complete this cell ###


bit_precision = 16
# out is array of size columns x len(o_nijg)

for out_tile in range(2):
    file = open('P2_Files/Tile' +  str(out_tile) + '/out.txt', 'w') #write to file
    file.write('#time0col7[msb-lsb],time0col6[msb-lst],....,time0col0[msb-lst]#\n')
    file.write('#time1col7[msb-lsb],time1col6[msb-lst],....,time1col0[msb-lst]#\n')
    file.write('#................#\n')
    for i in range(out.size(1)):  # time step
        for j in range(col_num): # col #
            out_bin = '{0:016b}'.format(0 if (round(out[col_num*(out_tile+1)-1-j,i].item()) < 0) else round(out[col_num*(out_tile+1)-1-j,i].item()))
            for k in range(bit_precision):
                file.write(out_bin[k])        
            #file.write(' ')  # for visibility with blank between words, you can use
        file.write('\n')
    file.close() #close file 

In [12]:
print(w_int[:,:,0])

tensor([[-2.,  1., -5.,  6.,  5.,  3., -2., -4., -4., -7., -3.,  6., -4., -3.,
          6., -7.],
        [ 3., -6.,  1., -5., -8.,  0.,  5.,  1., -8.,  5., -5., -3.,  1., -3.,
          7.,  5.],
        [ 5., -4.,  4., -6., -4.,  4.,  4.,  6.,  2., -7., -5.,  5., -1., -1.,
          5.,  3.],
        [-4.,  7.,  0., -5., -7.,  7., -2., -3.,  2.,  3.,  1.,  2., -6., -3.,
         -2.,  6.],
        [ 4., -1., -1.,  3., -3., -6.,  0., -4., -6., -1., -4.,  5.,  1.,  3.,
          1., -4.],
        [ 4.,  2.,  2.,  1.,  3.,  5., -2., -3.,  4., -1., -3., -8., -3., -6.,
         -6., -7.],
        [-7., -6., -4.,  4.,  5.,  7., -7., -1., -3.,  3.,  1., -8., -8.,  5.,
          5.,  7.],
        [-7., -8.,  0.,  2., -4.,  4., -3.,  2., -2., -8., -1., -4., -5., -1.,
         -5.,  6.],
        [ 3., -4.,  2.,  1.,  4.,  0., -7.,  5.,  5.,  0.,  1.,  2.,  1.,  7.,
          3.,  3.],
        [ 6., -3., -3., -6., -5., -3., -7., -1.,  3., -8., -3., -8., -3.,  5.,
          3., -3.],
        [-